<a href="https://colab.research.google.com/github/IsraelAbebe/Amharic-NLP/blob/master/Amharic-Word2Vec/Amharic_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Word Embeddings For Amharic Language

## Section 1.  [Using Tensorflow Examples.](https://colab.research.google.com/drive/1aTDUSy8wjA1J07oN2r4Oy12Rgd2yhHDa?authuser=1#scrollTo=V5wlseG_FlDW)
## Section 2.  [Using SkipGram Udacity Tutorials.]()


# [TF Examples](https://www.tensorflow.org/tutorials/text/word_embeddings)

In [0]:
%tensorflow_version 2.x
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import os
import unicodedata

tfds.disable_progress_bar()

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
embedding_layer = layers.Embedding(1000,10)

In [0]:
DIRECTORY_URL = "gdrive/My Drive/dataset/CACO"
FILE_NAMES = ['gdrive/My Drive/dataset/CACO/CACO_TEXT.txt']

In [0]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  


def unicode_to_ascii(s):
    return tf.strings.strip(''.join(c for c in unicodedata.normalize('NFD', s.numpy().decode('utf8')) if unicodedata.category(c) != 'Mn'))

# Text files
files = tf.data.Dataset.list_files('gdrive/My Drive/dataset/CACO/CACO_TEXT.txt', shuffle=True, seed=None)


labeled_data_sets = []

lines_dataset = tf.data.TextLineDataset(files, compression_type=None, buffer_size=None, num_parallel_reads=None)
labeled_dataset = lines_dataset.map(lambda line: labeler(tf.py_function(unicode_to_ascii, [line], tf.string),0))
labeled_data_sets.append(labeled_dataset)

In [0]:
BUFFER_SIZE = 50000
BATCH_SIZE = 128
TAKE_SIZE = 50000


all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

In [0]:
all_labeled_data

<MapDataset shapes: (<unknown>, ()), types: (tf.string, tf.int64)>

In [0]:
for ex,_ in all_labeled_data.take(5):
  print(ex.numpy().decode('utf8'))

አሁን ወደ ዋናው የጉዞ ፕሮግራም እንመለስ ።
ደብዳቤውን ካነበብክ በኋላ " ጓደኛዬ እኮ እንዲህ አለኝ ! " ብለህ በደስታ ለሌሎች ትናገር ይሆናል ።
የትምህርት ቤቶች እገዛና ድጋፍ ከፍተኛ መሆን እንዳለበትም አስተያየት ሰጥተዋል ።
አብደሻል ?
ደንብ ካልተከበረ የአዲስ አበባ ከተማን ዕድገት ማሳለጥም አይቻልም ።


In [0]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for ex,_ in all_labeled_data:
  some_tokens = tokenizer.tokenize(ex.numpy().decode('utf8'))
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

904412

In [0]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [0]:
example_text = next(iter(all_labeled_data))[0].numpy().decode('utf8')
print(example_text)

አሁን ወደ ዋናው የጉዞ ፕሮግራም እንመለስ ።


In [0]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[222004, 338681, 204775, 679989, 429849, 798820]


In [0]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy().decode('utf8'))
  return encoded_text, label

def encode_map_fn(text, label):
  return tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

all_encoded_data = all_labeled_data.map(encode_map_fn)

In [0]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([-1],[]))

In [0]:
train_batch, train_labels = next(iter(train_data))
train_labels.numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          14470624  
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 14,470,913
Trainable params: 14,470,913
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_data,
    epochs=10,
    validation_data=test_data, validation_steps=20)

Epoch 1/10
   2788/Unknown - 662s 238ms/step - loss: 0.0209 - accuracy: 1.0000

# [Skip-gram Word2Vec](https://github.com/IsraelAbebe/deep-learning-v2-pytorch/blob/master/word2vec-embeddings/Skip_Grams_Solution.ipynb)

### Loading Data

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
text_data_path = 'gdrive/My Drive/dataset/CACO/CACO_TEXT.txt'

with open(text_data_path) as f:
    text = f.read()

In [4]:
print(text[:200])

አሁን ወደ ዋናው የጉዞ ፕሮግራም እንመለስ ።
ደብዳቤውን ካነበብክ በኋላ " ጓደኛዬ እኮ እንዲህ አለኝ ! " ብለህ በደስታ ለሌሎች ትናገር ይሆናል ።
የትምህርት ቤቶች እገዛና ድጋፍ ከፍተኛ መሆን እንዳለበትም አስተያየት ሰጥተዋል ።
አብደሻል ?
ደንብ ካልተከበረ የአዲስ አበባ ከተማን ዕድገት ማሳለጥም አይቻልም ።
የ


### Pre-processing

In [0]:
import re
from collections import Counter

def preprocess(text):

    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('።', ' <PERIOD> ')
    text = text.replace('፣', ' <PERIOD> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' ')
    text = text.replace('፤','<SEMICOLON>')
    text = text.replace('…',' <HYPHENS> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

In [11]:
words = preprocess(text)
print(words[:500])

['አሁን', 'ወደ', 'ዋናው', 'የጉዞ', 'ፕሮግራም', 'እንመለስ', '<PERIOD>', 'ደብዳቤውን', 'ካነበብክ', 'በኋላ', '<QUOTATION_MARK>', 'ጓደኛዬ', 'እኮ', 'እንዲህ', 'አለኝ', '<EXCLAMATION_MARK>', '<QUOTATION_MARK>', 'ብለህ', 'በደስታ', 'ለሌሎች', 'ትናገር', 'ይሆናል', '<PERIOD>', 'የትምህርት', 'ቤቶች', 'እገዛና', 'ድጋፍ', 'ከፍተኛ', 'መሆን', 'እንዳለበትም', 'አስተያየት', 'ሰጥተዋል', '<PERIOD>', 'አብደሻል', '<QUESTION_MARK>', 'ደንብ', 'ካልተከበረ', 'የአዲስ', 'አበባ', 'ከተማን', 'ዕድገት', 'አይቻልም', '<PERIOD>', 'የመማጸኛ', 'ከተሞች', '<LEFT_PAREN>', '9', '34', '<RIGHT_PAREN>', 'ከዚህ', 'በተጨማሪም', '<PERIOD>', 'የእርሻ', 'ስራ', 'ከጀመሩ', 'ወዲህ', 'በማኅበር', 'ተደራጅተው', '100ሺ', 'ብር', 'ብድር', 'ወስደው', 'በገቢ', 'ማስገኛ', 'ስራዎች', 'ላይ', 'ተሰማርተዋል', '<PERIOD>', 'ባለ', 'ድርሻ', 'አካላትንና', 'የክልሉን', 'ማህበረሰብ', 'ፍላጎት', 'መሰረት', 'ያደረጉ', 'የምርምር', 'ፕሮፖዛሎች', 'የዩኒቨርሲቲውን', 'ዌብ', 'ሳይት', 'በመጠቀም', 'በተደረገው', 'የፕሮፖዛል', 'ጥሪ', 'መሰረት', 'በአጠቃላይ', '85', 'የምርምር', 'ፕሮፖዛሎች', 'ቀርበዋል', '<PERIOD>', 'ከተስፋ', 'አስቆራጩ', 'የኤምባሲው', 'ምላሽ', 'በኋላም', 'የሚመጣውን', 'በፀጋ', 'ለመቀበል', 'ወስነው', 'ጥቅምት', 'ሃያ', 'አራትን', 'መጠባበቅ', 'ጀመሩ', '<PERIOD>', 'ሌላው', 'የአከባቢው', 'ነዋሪ', 'አርሶ', 'አ

In [12]:
# print some stats about this word data
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words)))) # `set` removes any duplicate words

Total words in text: 20762351
Unique words: 172089


### Dictionaries

In [0]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: Two dictionaries, vocab_to_int, int_to_vocab
    """
    word_counts = Counter(words)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [14]:
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[44, 6, 663, 4455, 594, 13201, 0, 12122, 102705, 33, 1, 4996, 277, 45, 600, 15, 1, 1373, 1877, 940, 38683, 96, 0, 292, 188, 20811, 144, 47, 115, 5906]


### Subsampling

In [15]:
from collections import Counter
import random
import numpy as np

threshold = 1e-5
word_counts = Counter(int_words)
#print(list(word_counts.items())[0])  # dictionary of int_words, how many times they appear

total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
# discard some frequent words, according to the subsampling equation
# create a new list of words for training
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

print(train_words[:30])

[13201, 12122, 102705, 600, 38683, 292, 20811, 5906, 386, 1564, 138743, 1166, 76088, 5565, 410, 30235, 393, 5174, 17092, 3285, 23153, 9207, 9833, 16211, 489, 61401, 4854, 948, 1360, 49356]


### Making batches

In [0]:

def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [17]:
# test your code!

# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [1, 2, 3, 4, 6, 7, 8, 9]


In [0]:

def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [19]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 1, 1, 2, 2, 3, 3, 3]
y
 [1, 2, 0, 2, 1, 3, 0, 1, 2]


### Building the graph

In [0]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## SkipGram model

In [0]:
import torch
from torch import nn
import torch.optim as optim

In [0]:

class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)
        
        return log_ps

In [0]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim=300 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 500
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(train_words, 64):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        
        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

ጊዜ | ሰገል, እኝ, ላብራቶሪው, ሲያጋጥሙ, እየተገነቡና
ዓመት | አስተካካዩ, ጊዜያዊውን, አራሜክስ, ከአላስፈላጊ, መድፈኞች
ከዚህ | መምህራኑ, ውላጅ, ይከፈላል, መጫሚያ, የሚተረጎመው
እንዲሁም | የሚመጡትም, በማዕድናትና, የነጡ, ዮሐንስም, አትሌቲክስና
<COMMA> | አጣቦ, እያቀረብን, ከኪሎ, ጥርጣሬም, አደረግናቸው
ችግር | የተቻለበትን, ያስመዘገቧቸው, ምኒልክን, በከፍታ, ቁጣህም
እንዴት | እከንፋለሁ, በስነልቦናም, ጠንቃቃ, መታሰርን, ዘሮቹም
እኔ | ሃኪሞች, እንደማይሳካ, ዘርፈብዙ, ወርሶ, አንሥቶ
በአንዳንድ | ግፈኞች, በገለጽኩት, በሶሪያም, ከዩኒቨርስቲው, ጥፋተኝነታቸውን
መስሪያ | በማጥበቅ, የዓዲግራት, ከረዓብ, እንድናተኩር, ያቋቁማሉ
መሣሪያ | ቀጣዩንና, አይቼህ, ተረግዞ, ምዝገባውን, የሚመለከተን
፦ | ኢምፓየር, ለታሪክም, መገልገል, በኢሉ, ተሳትፈውበት
ሥራውን | ፋካልቲ, ቆርኬዎች, ሰላምታዎች, መናው, ይወልዱ
በደንብ | የእሸት, የተጐዳው, ግዛቱን, ሰሙ, እያጋጠሟቸው
አብዛኞቹ | ከተዘረጉ, የሚያስደስተኝ, ማጽናናት, ካልገዛሁ, አላስቆም
ቋሚ | በመረጋገጡ, ከወገብ, ለመላእክቱ, ጥሰቱ, ሳንድ
...
ደግሞ | ያልተፈቀዱ, event, የተሳካላቸው, ማውገዙ, ኮስሞቲክስ
ሥራ | ስለሌሎች, ሚኒስትሮችን, ሉሲዎቹን, የሚያሰለጥናቸው, የመፍትሄው
አለ | በፋቁሄ, የማማረር, ተመቻችቶ, ፍሪጅ, ለሚያከናውኗቸው
ሆኖ | መድሐኒቶች, መዝጊያዎቹን, ቀርቧልና, ለክብረ, አሉባልተኞች
ናቸው | የተጋለጠውን, ኅብረትም, የመስመር, አትታወቅም, የሚዘክር
በጣም | የሚያመለከት, ቆዩ, መዋሉ, የማስፋት, መለያ
ኢትዮጵያ | አነባለሁ, ፖሊሲያቸው, የሚያስከፋ, ፀሐዬ, አንቀላፍቶ
ሰዎች | ተሰላችቶ, ከሐኪሟ, የተመረቀው, የተፈለገው, የሚገናኙበትን

# Visualizing the word vectors

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [0]:
embeddings = model.embed.weight.to('cpu').data.numpy()

In [0]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [0]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)